<a href="https://colab.research.google.com/github/eswens13/style_transfer/blob/dev/erik/music_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mido

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import os
import mido

DRIVE_DIR = '/content/drive/My Drive/deep_learning/midis'
midi_bytes = []
filepath = os.path.join(DRIVE_DIR, 'beethoven_beethoven_opus10_3.mid')
with open(filepath, 'rb') as infile:
  midi_bytes = infile.read()

print("Read {} bytes".format(len(midi_bytes)))
parser = mido.Parser()
for b in midi_bytes:
  parser.feed_byte(b)

stopped = False
while not stopped:
  msg = parser.get_message()
  if not msg:
    stopped = True
  else:
    print(msg)



Read 27680 bytes
pitchwheel channel=0 pitch=2637 time=0
reset time=0
reset time=0
polytouch channel=9 note=32 value=50 time=0
reset time=0
reset time=0
reset time=0
reset time=0
reset time=0
reset time=0
reset time=0
reset time=0
program_change channel=0 program=0 time=0
note_on channel=0 note=60 velocity=40 time=0
note_off channel=1 note=112 velocity=48 time=0
control_change channel=0 control=64 value=127 time=0
note_on channel=0 note=47 velocity=0 time=0
control_change channel=0 control=64 value=0 time=0
reset time=0
note_on channel=0 note=48 velocity=0 time=0
reset time=0
note_on channel=0 note=65 velocity=38 time=0
reset time=0
note_on channel=0 note=63 velocity=45 time=0
note_off channel=3 note=96 velocity=51 time=0
reset time=0
note_on channel=0 note=60 velocity=40 time=0
note_off channel=1 note=112 velocity=48 time=0
control_change channel=0 control=64 value=127 time=0
note_on channel=0 note=47 velocity=0 time=0
control_change channel=0 control=64 value=0 time=0
reset time=0
not